# Modelo para previsão do score de um filme no ranking do IMDB

### Base de dados:
https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset

In [ ]:
#Bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import itertools
import pickle
import os

In [ ]:
#Definindo diretórios 
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
ANALISE_DIR = os.path.join(BASE_DIR, 'analise')

#Checando ambiente

In [ ]:
conda info --env

In [ ]:
#Função para comparar os shapes para verificar % da base que ainda resta
def compara_shapes(shape_end,shape_init):
    """Função que retorna % de comparação entre shapes, onde paramentro 1 é divisor e parametro 2 é dividendo"""
    compara_apos_nan = round((shape_end/shape_init)*100,2)
    compara_apos_nan = compara_apos_nan.rename({0:'%Linhas restantes',1:'%Colunas restantes'})
    return compara_apos_nan

## Análise Exploratória

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, 'movie_metadata.csv'))

In [ ]:
df.head()

In [ ]:
#Salvo meu shape inicial
shape_inicial = pd.Series(df.shape)

In [ ]:
df.dtypes

In [ ]:
#Verifica valores da coluna cor
df['color'].value_counts()

In [ ]:
#Removendo coluna de link, cor do filme, linguagem e país, pois as mesmas não possuem relevância para o modelo
df.drop(columns=['movie_imdb_link','color','country','language'], axis=1, inplace=True)

In [ ]:
list(df.columns)

### Valores missing

In [ ]:
#Vendo colunas que possuem NaN
df.isna().any()

In [ ]:
#Crio dicionário com minhas colunas e a soma das suas linhas nulas e uma lista vazia
nan_colunas = dict(df.isna().sum())
nan_lista_deletar = list()

In [ ]:
# Pego todas as colunas que possuem menos de 154 linhas nulas e adiciono na lista criada
for key, value in nan_colunas.items():
    if value <= 153 and value != 0:
        nan_lista_deletar.append(key)
        
nan_lista_deletar

In [ ]:
# dropo meu os NaN utilizando lista das colunas acima
df.dropna(subset=nan_lista_deletar, axis=0, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
#Salvo meu shape após exclusão de NaN
shape_nan = pd.Series(df.shape)

In [ ]:
#Chamo função que compara Shapes
compara_shapes(shape_nan,shape_inicial)

In [ ]:
#Ao ver que a maioria dos Filmes é R (Livre)
df['content_rating'].value_counts()

In [ ]:
#Iremos substituir os NaN por R
df['content_rating'].fillna('R', inplace=True)

In [ ]:
#Ao ver que a distribuição está mais para a direita
df['aspect_ratio'].value_counts()

In [ ]:
#Iremos substituir os NaN pela mediana
df['aspect_ratio'].fillna(df['aspect_ratio'].median(), inplace=True)

In [ ]:
#E assim da mesma forma para o Faturamento e para o Orçamento
df['budget'].fillna(df['budget'].median(), inplace=True)
df['gross'].fillna(df['gross'].median(), inplace=True)

In [ ]:
df.isna().sum()

### Valores Duplicados

In [ ]:
#Soma dos valores duplicados
df.duplicated().sum()

In [ ]:
#Dropo valores duplicados e salvo shape
df.drop_duplicates(inplace=True)
shape_duplicated = pd.Series(df.shape)

In [ ]:
#Chamo função que compara Shapes
compara_shapes(shape_duplicated,shape_inicial)

### Criando Váriaveis

In [ ]:
#Variavel Lucro criada pela subtração do orçamento pelo valor arrecadado bruto
df['profit'] = df['budget'].sub(df['gross'], axis=0)

In [ ]:
#Crio % do Lucro em relação ao valor bruto
df['profit_percentage'] = (df['profit']/df['gross'])*100 

In [ ]:
df.head()

In [ ]:
#Salvando DataSet em csv
df.to_csv(os.path.join(ANALISE_DIR, 'dados_imdb_analiseexpl.csv'), index=False)

### Alguns Insights

In [ ]:
#Aqui podemos ver que o orçamento é relacionado a nota do filme
ggplot(aes(x='imdb_score', y='profit'),data=df) +\
    geom_line() +\
    stat_smooth(colour='blue', span=1)

In [ ]:
#Aqui podemos ver que o número de likes no facebook é relacionado a nota do filme
(ggplot(df)+\
    aes(x='imdb_score', y='movie_facebook_likes') +\
    geom_line() +\
    labs(title='Nota no IMDB x Likes no facebook', x='Nota no IMDB', y='Likes no facebook')
)

In [ ]:
#Correlaciona qual a nota do imdb com o ator principal do filme (nos 20 melhores filmes)
plt.figure(figsize=(10,8))
df = df.sort_values(by='imdb_score', ascending=False)
df2 = df.head(20)
ax = sns.pointplot(df2['actor_1_name'], df2['imdb_score'], hue=df2['movie_title'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=40, ha='right')
plt.tight_layout()
plt.show()


### Preparação para o modelo

In [ ]:
#Retirando algumas colunas que não possuem importancia
df.drop(columns=['director_name','actor_1_name','actor_2_name',
                 'actor_3_name','plot_keywords', 'movie_title',
                'genres', 'profit','profit_percentage'], axis=1, inplace=True)

In [ ]:
#Comparação das variáveis entre si
corr = df.corr()
sns.set_context('notebook', font_scale=1.0, rc={'lines.linewidt':2.5})
plt.figure(figsize=(13,7))
#Cria mascara para ver somente as variáveis correlatas
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask,1)]=True
a = sns.heatmap(corr, mask=mask, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
rotx = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
#Variaveis de likes no facebook são muito correlatas, precisamos modificar
df['other_actors_facebook_likes'] = df['actor_2_facebook_likes'] + df['actor_3_facebook_likes']
df.drop(columns=['actor_2_facebook_likes','actor_3_facebook_likes', 'cast_total_facebook_likes'], axis=1, inplace=True)

In [ ]:
#Variaveis de review são muito correlatas, precisamos modificar
df['critic_review_ratio'] = df['num_critic_for_reviews']/df['num_user_for_reviews']
df.drop(columns=['num_critic_for_reviews','num_user_for_reviews'], axis=1, inplace=True)

In [ ]:
#Agora temos todas as variaveis com menos de 65% de correlação, podemos continuar
corr = df.corr()
sns.set_context('notebook', font_scale=1.0, rc={'lines.linewidt':2.5})
plt.figure(figsize=(13,7))
#Cria mascara para ver somente as variáveis correlatas
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask,1)]=True
a = sns.heatmap(corr, mask=mask, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
rotx = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
#Tranformando variavel numerica para categorizar a nota de 1 a 4
df['imdb_binned_score'] = pd.cut(df['imdb_score'], bins=[0,4,6,8,10], right=True, labels=False)+1

In [ ]:
#Tranformando faixa etaria em várias colunas "booleanas" de acordo com valor
df = pd.get_dummies(data=df, columns=['content_rating'], prefix=['content_rating'], drop_first=True)
shape_modelo = pd.Series(df.shape)

In [ ]:
#Função para comparar shape
compara_shapes(shape_modelo,shape_inicial)

In [ ]:
#Salvando DataSet em csv
df.to_csv(os.path.join(ANALISE_DIR, 'dados_imdb_analisemodelo.csv'), index=False)

### Separando os dfs de treino e teste

In [ ]:
#Separando colunas para o modelo
colunas_df = list()
for coluna in df.columns:
    if coluna[0:4] != 'imdb':
        colunas_df.append(coluna)

In [ ]:
#Criando df para modelagem sem as colunas imdb_score e imdb_binned_score
X = pd.DataFrame(columns=colunas_df, data=df)

In [ ]:
y= pd.DataFrame(columns=['imdb_binned_score'], data=df)

In [ ]:
#Criando treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#Normalizando os dados
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Modelo
### Regressão Logística

In [ ]:
#Criando modelo
logit = LogisticRegression(verbose=1, max_iter=1000)
logit.fit(X_train,np.ravel(y_train,order='C'))
y_pred = logit.predict(X_test) 

In [ ]:
#Verificando acuracia
cnf_matrix = metrics.confusion_matrix(y_test,y_pred)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    '''
    This function print and plots the confusion matrix.
    Normalization can be applied by setting "normalize=True"
    '''
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j], fmt),
                horizontalalignment = 'center',
                color='white' if cm[i,j] > thresh else 'black')
    
    plt.ylabel('True label')
    plt.xlabel('Predict label')
    
    plt.tight_layout()
    

In [ ]:
#Ploto Matriz de confusão
plot_confusion_matrix(cnf_matrix,classes=['1','2','3','4'], title='Confusion Matrix not normalized Score IMDB')

In [ ]:
#Aqui podemos ver que o df está desbalanceado, por isso do resultado que não acerta em 1 e acerta mais em 3
df['imdb_binned_score'].value_counts()

In [ ]:
#Report de precisão das classes
print(metrics.classification_report(y_test,y_pred,target_names=['1','2','3','4']))

In [ ]:
#Salvando modelo
modelo_treinado='modelo_imdb.sav'
pickle.dump(logit,open(modelo_treinado,'wb'))

In [ ]:
#Utilizando o modelo
modelo_carregado = pickle.load(open(modelo_treinado,'rb'))
modelo_carregado.predict([X_test[0]])